In [1]:
import pandas as pd

In [2]:
test_data = pd.read_csv(r"C:\Users\ruchit\Desktop\datasets\twitter\twitter_test.csv")
train_data= pd.read_csv(r"C:\Users\ruchit\Desktop\datasets\twitter\twitter.csv")

In [3]:
train_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
#only useful feature from the whole dataframe is the 'text'
#rest all dont have much affect on the sentiment as a whole

In [5]:
x_data = train_data['text']
y_data=train_data['airline_sentiment']

In [6]:
x_test_data  = test_data['text']

# Cleaning The Data

In [7]:
from nltk.corpus import stopwords

In [8]:
stop = stopwords.words('english')

In [9]:
import string
punc = list(string.punctuation)

In [35]:
#making stop word list to clean our dataset
stop+=punc
stop[:5]

['i', 'me', 'my', 'myself', 'we']

In [11]:
from sklearn.model_selection import train_test_split

In [13]:
#splitting the training data given to us so that we can compare models
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,random_state=0)

In [17]:
#Importing our vectorizer

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
#initialising the vectorizer

In [36]:
tf_idf = TfidfVectorizer(max_features=3000,ngram_range=(1,2),stop_words=stop,max_df=.8,lowercase=True,use_idf=True)

In [37]:
#construction of our features aka -> sparse matrix
train_feat = tf_idf.fit_transform(x_train)
test_feat = tf_idf.transform(x_test)

In [38]:
#trying naive bayes algorithm
from sklearn.naive_bayes import MultinomialNB
bayes = MultinomialNB(alpha=0.4)
bayes.fit(train_feat, y_train)

MultinomialNB(alpha=0.4, class_prior=None, fit_prior=True)

In [39]:
#score on data splitted from training one itself
bayes.score(test_feat,y_test)

0.7555555555555555

In [40]:
#applying grid search to find the most optimal params.
from  sklearn.model_selection import GridSearchCV
clf = MultinomialNB()
grid = {"alpha" :[0.1,0.2,0.3,0.4,0.5,0.6,0.7]}
gsc = GridSearchCV(clf, grid) 
gsc.fit(test_feat, y_test)

GridSearchCV(cv=None, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
gsc.best_estimator_

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [42]:
#making our test data take form into desirable sparse matrix and prediciting the outcomes on the same
pred_features = tf_idf.transform(x_test_data)
y_pred = bayes.predict(pred_features)

In [43]:
import  numpy as  np
np.savetxt("output3.csv" , y_pred, fmt='%s')

In [44]:
from sklearn.neural_network import MLPClassifier

In [60]:
clf = MLPClassifier(hidden_layer_sizes=(100))

In [61]:
clf.fit(train_feat,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=100, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [62]:
clf.score(test_feat,y_test)

0.7289617486338797